# STUDIO ERRORI SPETTROFLUORIMETRO

per lo studio degli errori prendo gli spettri, li normalizzo rispetto al parametro massimo, interpolo, e calcolo errore percentuale

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

In [3]:
filename = 'data/assorbanza_rodamina_concentrazioni_tot.csv'

header = ['λ 5','I 5','λ 4','I 4','no λ 4','no I 4','λ 3','I 3','λ 2','I 2','λ 1','I 1','λ 0.8','I 0.8','λ 0.6','I 0.6','λ 0.4','I 0.4','λ 0.2','I 0.2','λ 0.1','I 0.1']

data = pd.read_csv(filename,sep=',',header=1,nrows=117)
data = data.iloc[:, :-1] # drop last column
data.columns = header

In [5]:
filename = './data/concentrazioni_rodamina.csv'

volumi = pd.read_csv(filename,sep=',')
volumi = volumi[['Vi (muL)','Vf (muL)']]
# Display the DataFrame horizontally
display(volumi.T)

A0, sigmaA0 = 0.4800416799849726, 0.0003353431037236748
ASS = [A0]
SIGMA_ASS = [sigmaA0]
sigmaV = 1 # μL, comodo perchè si ha sempre a che fare con rapporti
for idx, row in enumerate(volumi.iterrows()):
    vi, vf = row[1]['Vi (muL)'], row[1]['Vf (muL)']
    An = ASS[idx] * vi / vf
    sAn = An * np.sqrt((sigmaV/vi)**2 + (sigmaV/vf)**2 + (SIGMA_ASS[idx]/ASS[idx])**2) 
    ASS.append(An)
    SIGMA_ASS.append(sAn)

ASS = np.array(ASS)
SIGMA_ASS = np.array(SIGMA_ASS)

,0,1,2,3,4,5,6,7,8
Vi (muL),2210,2252,2000,1511,2423,2251,2002,1497,1504
Vf (muL),3008,3015,3015,3016,3010,3002,3000,2994,3003


In [6]:
fig = go.Figure()

WAVELENGHTS = ['λ 5', 'λ 4', 'λ 3', 'λ 2', 'λ 1', 'λ 0.8', 'λ 0.6', 'λ 0.4', 'λ 0.2', 'λ 0.1']
INTENSITIES = ['I 5', 'I 4', 'I 3', 'I 2', 'I 1', 'I 0.8', 'I 0.6', 'I 0.4', 'I 0.2', 'I 0.1']

for λ_col, I_col, a in zip(WAVELENGHTS, INTENSITIES, ASS):
    fig.add_trace(go.Scatter(x=data[λ_col], y=data[I_col],
                    mode='lines',
                    name=f'{np.round(a, 2)}'))
    
# fig.add_trace(go.Scatter(x=fondo['λfondo'],y=fondo['Ifondo'],
#                     mode='lines',
#                     name=f'fondo'))

fig.update_layout(
                  xaxis_title='Wavelength (nm)',
                  yaxis_title='Intensity (a.u.)',
                  height=600,
                  width=800,
                  yaxis=dict(range=[0, 1000]),
                  legend=dict(x=0.9, y=0.9),
                  font=dict(size=14))

fig.write_html(dir_path + r"\html\emission_spectrum.html")
fig.write_image(dir_path + r"\images\emission_spectrum.png")
fig.show()

plot spettri normalizzati

In [7]:
SIGMA_LAMBDA = 1.5

def max_fit_parabolic(x, λcenter, a, IMAX): # -a*(x-λcenter)**2 + IMAX
    return -a*(x-λcenter)**2 + IMAX

λcenter_list = []
a_list = []
IMAX_list = []
error_λcenter_list = []
error_a_list = []
error_IMAX_list = []
sigmay = [] # contiene l'errore propagato con x
for λ_col, I_col in zip(WAVELENGHTS, INTENSITIES):
    Λ = data[λ_col]
    I = data[I_col]
    Λmaxcenter = I.idxmax()

    λcenter = Λ[Λmaxcenter] # guess
    IMAX = I.max() # guess
    a = 1 # guess

    xrangemax = Λ[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    yrangemax = I[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    # ---- per considerare errore su lambda di 1.5nm
    # iterazione 0
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX])
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))
    # iterazione 1
    dydl = np.abs(-2*a*(λcenter-xrangemax)) # derivata prima rispetto a λ
    sy = dydl*SIGMA_LAMBDA
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX],sigma=sy)
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))
    # iterazione 2
    d2ydl2 = np.abs(-2*a) # derivata seconda rispetto a λ
    sy = np.sqrt((dydl*SIGMA_LAMBDA)**2 + (0.5*d2ydl2*SIGMA_LAMBDA**2)) # errore propagato
    popt, pcov = curve_fit(max_fit_parabolic, xrangemax, yrangemax, p0=[λcenter, a, IMAX],sigma=sy)
    λcenter, a, IMAX = popt
    error_λcenter, error_a, error_IMAX = np.sqrt(np.diag(pcov))

    λcenter_list.append(λcenter)
    a_list.append(a)
    IMAX_list.append(IMAX)
    error_λcenter_list.append(error_λcenter)
    error_a_list.append(error_a)
    error_IMAX_list.append(error_IMAX)
    sigmay.append(sy)

# to numpy
λcenter_list = np.array(λcenter_list)
a_list = np.array(a_list)
IMAX_list = np.array(IMAX_list)
error_λcenter_list = np.array(error_λcenter_list)
error_a_list = np.array(error_a_list)
error_IMAX_list = np.array(error_IMAX_list)
sigmay = np.array(sigmay)

print('Tabella con i fit di tutti i picchi delle parabole')
max_fit_parabolic_dataframe = pd.DataFrame({'Absorbance': ASS, 'λcenter': λcenter_list, 'a': a_list, 'IMAX': IMAX_list, 'error_λcenter': error_λcenter_list, 'error_a': error_a_list, 'error_IMAX': error_IMAX_list})
display(max_fit_parabolic_dataframe)


Tabella con i fit di tutti i picchi delle parabole


,Absorbance,λcenter,a,IMAX,error_λcenter,error_a,error_IMAX
0,0.480042,554.044219,2.442431,940.693325,0.274804,0.443226,1.342285
1,0.352690,553.742124,2.156915,819.968625,0.285710,0.435038,1.546347
2,0.263436,552.758972,1.744154,718.450202,0.067448,0.081499,0.254055
3,0.174750,551.865053,1.027011,492.753337,0.074358,0.052306,0.178710
4,0.087549,551.494151,0.503627,238.703070,0.147158,0.053501,0.250304
5,0.070475,551.279005,0.386982,170.262114,0.137228,0.036855,0.180003
6,0.052845,551.307117,0.243386,117.772112,0.080942,0.013286,0.070467
7,0.035265,551.090509,0.101588,52.545366,0.128419,0.009120,0.057811
8,0.017633,550.856778,0.031331,19.321189,0.203719,0.004558,0.033509
9,0.008831,550.925065,0.024793,11.330715,0.231624,0.004218,0.036550


In [8]:
center = np.mean(λcenter_list)
sigma = np.std(λcenter_list)
print('λcenter =', final_val(center, sigma, udm='nm'))
print('NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato')

λcenter = 551.94 ± 1.11 nm
NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato


In [9]:
# plotting normalized spectrum (kind of, in the sense that we are not normalizing the area under the curve, but the maximum value of the spectrum)
delta = np.array(λcenter_list) - np.mean(λcenter_list)  # Calculate the average value of λcenter

plt.figure(figsize=(10, 6))

fig = go.Figure()

for λ_col, I_col, c, d in zip(WAVELENGHTS, INTENSITIES,ASS,delta):
    max = data[I_col].max()
    centered_wavelengths = data[λ_col] - d  # Center the wavelengths around the average λcenter
    ydata = data[I_col]*100/ max

    # solo per plottare interpolazione, non utile

    # f = interp1d(centered_wavelengths,ydata)
    # x = np.linspace(np.min(centered_wavelengths),np.max(centered_wavelengths))
    # y = f(x)

    # fig.add_trace(go.Scatter(x=x,y=y,
    #                 mode='lines',
    #                 name=f'{np.round(c,2)}'))

    fig.add_trace(go.Scatter(x=centered_wavelengths,y=ydata,
                    mode='lines',
                    name=f'{np.round(c,2)}'))

fig.update_layout(
                  xaxis_title='Wavelength (nm)',
                  yaxis_title='Intensity (%)',
                  height=600,
                  width=800,
                  yaxis=dict(range=[0, 100]),
                  legend=dict(x=0.88, y=1),
                  font=dict(size=14))

fig.write_html(dir_path +r"\html\normalized_centered_emission_spectrum.html")
fig.write_image(dir_path + r"\images\normalized_centered_emission_spectrum.png")
fig.show()

<Figure size 1000x600 with 0 Axes>

In [10]:
delta = np.array(λcenter_list) - np.mean(λcenter_list)  # Calculate the average value of λcenter

interpolation_functions = []

for λ_col, I_col, c, d in zip(WAVELENGHTS, INTENSITIES,ASS,delta):
    max = data[I_col].max()
    centered_wavelengths = data[λ_col] - d  # Center the wavelengths around the average λcenter
    ydata = data[I_col]*100/ max
    f = interp1d(centered_wavelengths,ydata)
    interpolation_functions.append(f)

x = np.arange(540,640)
y = np.array(list(map(lambda F: F(x),interpolation_functions)))

mean_y = np.mean(y, axis=0)
std_y = np.std(y, axis=0)
perc_error = (std_y / mean_y) * 100

print('Errore percentuale medio: ',np.round(np.mean(perc_error),2),'%')
# Display the results
# print("Mean values:", mean_y)
# print("Standard deviation values:", std_y)
# print("Percent error: ", perc_error)

Errore percentuale medio:  1.5 %


In [31]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=x, y=perc_error, mode='lines', name='Percent Error'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=x, y=mean_y, mode='lines', name='Mean Intensity'),
    secondary_y=True,
)


# Add figure title
fig.update_layout(
    height=600,
    width=800,
    font=dict(size=14),
    showlegend=False,
)

# Set x-axis title
fig.update_xaxes(title_text="Wavelength (nm)", showgrid=False)

# Set y-axes titles and remove grid
fig.update_yaxes(title_text="<span style='color:#636EFA'>Percent Error (%)</span>", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="<span style='color:#EF553B'>Intensity (%)</span>", secondary_y=True, showgrid=False)

fig.show()